# Data Storytelling - JOLTS
#### _A look at US Job Openings data over time_
***
_Michael Garber_

# Table of Contents
> 1. Introduce dataset
2. Pick Audience
3. EDA
4. Create Narrative
5. Presentation

## Introduction

- I have chosen to analyze the the ***JOLTS*** dataset

- "The Job Openings and Labor Turnover Survey (JOLTS) program of the Bureau of Labor Statistics (BLS) produces monthly and annual estimates of job openings, hires, and separations for the nation."

- Goal: share the story told by the recent trends found within. I will specifically focus on the job openings to understand the labor demands.

- Data Set
> Info URL - https://www.bls.gov/jlt/jltover.htm \
Data URL - https://download.bls.gov/pub/time.series/jt/jt.data.2.JobOpenings \
Data Definitions - https://download.bls.gov/pub/time.series/jt/jt.txt

## Pick Audience

- I have chosen to target a ***non-technical audience***.
    - _I will take care to make the findings accessible and interesting._

## EDA
> What can we learn about the data?

- Can I count something interesting?
- Can I find trends (e.g. high, low, increasing, decreasing, anomalies)?
- Can I make a bar plot or a histogram?
- Can I compare two related quantities?
- Can I make a scatterplot?
- Can I make a time-series plot?
- Looking at the plots, what are some insights I can make?
- Can I see any correlations?
- Is there a hypothesis I can - and should - investigate further?
- What other questions are the insights leading me to ask?

## Create Narrative
Narrative here...

## Presentation
presentation images and text in this section...

> The questions you asked? \
The trends you investigated? \
The resulting visualizations and conclusions?